In [2]:
#importing dependencies
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np

%matplotlib inline
mpl.rcParams['figure.figsize']=(16,10)
pd.set_option('display.max_rows',500)

import plotly.graph_objects as go

# DATA LOADING

In [3]:
#load our dataset and parse the dates right at the beginning
df_analyse=pd.read_csv('../data/processed/COVID_small_flat_table.csv',sep=';',parse_dates=[0])
#sorting the dates in the data in ascending order
df_analyse.sort_values('date',ascending=True).tail()

date     Italy        US     Spain     India   Germany
871 2022-06-11  17634065  85506177  12478994  43222017  26803867
872 2022-06-12  17653375  85515529  12478994  43230101  26809245
873 2022-06-13  17664043  85632808  12478994  43236695  26915085
874 2022-06-14  17703887  85758638  12515127  43245517  27007429
875 2022-06-15  17736696  85941735  12515127  43257730  27096571

# Helper Functions

In [10]:
def quick_plot(x_in, df_input,y_scale='log',slider=False):
    """ Quick basic plot for quick static evaluation of a time series
    
        you can push selective columns of your data frame by .iloc[:,[0,6,7,8]]
        
        Parameters:
        ----------
        x_in : array 
            array of date time object, or array of numbers
        df_input : pandas dataframe 
            the plotting matrix where each column is plotted
            the name of the column will be used for the legend
        scale: str
            y-axis scale as 'log' or 'linear'
        slider: bool
            True or False for x-axis slider
    
        
        Returns:
        ----------
        
    """
    fig = go.Figure()                                                              #plotly figure

    for each in df_input.columns:                                                  #iterate through each column
        fig.add_trace(go.Scatter(
                        x=x_in,                                                     #x-scale
                        y=df_input[each],                                            #to push columns as per our need
                        name=each,
                        opacity=0.8))
    
    fig.update_layout(autosize=True,                                                #just to have a nice plot for visulaization
        width=1024,
        height=768,
        font=dict(
            family="PT Sans, monospace",
            size=18,
            color="#7f7f7f"
            )
        )
    fig.update_yaxes(type=y_scale),
    fig.update_xaxes(tickangle=-45,
                 nticks=20,
                 tickfont=dict(size=14,color="#7f7f7f")
                )
    if slider==True:
        fig.update_layout(xaxis_rangeslider_visible=True)
    fig.show()

In [44]:
#checking the function
quick_plot(df_analyse.date,       #x axis as dates
           df_analyse.iloc[:,1:], #y_axis as rest of the countries exceept dates so starting from 2nd column
           y_scale='log',
           slider=True)

In [45]:
#Different countries have different slopes so we need to keep some threshold so that all countries would have identical slope
compare_list=[]
threshold=10
for pos,country in enumerate(df_analyse.columns[1:]):
      #compare each country larger than threshold
    compare_list.append(np.array(df_analyse[country][df_analyse[country]>threshold]))

In [46]:
#converting list into a dataframe and assign it to a new variable
pd_sync_timelines=pd.DataFrame(compare_list,index=df_analyse.columns[1:]).T

In [47]:
#adding date column to this dataframe
pd_sync_timelines['date']=np.arange(pd_sync_timelines.shape[0])

In [48]:
pd_sync_timelines.head()

Italy    US  Spain  India  Germany  date
0   20.0  11.0   13.0   28.0     12.0     0
1   62.0  11.0   15.0   30.0     12.0     1
2  155.0  11.0   32.0   31.0     12.0     2
3  229.0  12.0   45.0   34.0     12.0     3
4  322.0  12.0   84.0   39.0     13.0     4

In [49]:
quick_plot(pd_sync_timelines.date,
           pd_sync_timelines.iloc[:,:-1],
           y_scale='log',
           slider=True)

# Doubling Rate

$N(t)=N_0*2^{t/T}$

Here N0 is the  base population and t is the rate at which doubling happens and T is the time period for which doubling occurs

In [50]:
#creating a function call for doubling rate
def doubling_rate(N_0,t,T_d):
    return N_0*np.power(2,t/T_d)

In [59]:
max_days= 40

norm_slopes={
    #'doubling every day':doubling_rate(100,np.arange(10),1),
    'doubling every two days':doubling_rate(100,np.arange(max_days),2),
    'doubling every 4 days':doubling_rate(100,np.arange(max_days),4),
    'doubling every 10 days':doubling_rate(100,np.arange(max_days),10),
}


In [60]:
#we can join this norm_slopes to our dataframe by concatenation
pd_sync_timelines_w_slope=pd.concat([pd.DataFrame(norm_slopes),pd_sync_timelines], axis=1)

In [61]:
pd_sync_timelines_w_slope

doubling every two days  doubling every 4 days  doubling every 10 days  \
0                 100.000000             100.000000              100.000000   
1                 141.421356             118.920712              107.177346   
2                 200.000000             141.421356              114.869835   
3                 282.842712             168.179283              123.114441   
4                 400.000000             200.000000              131.950791   
..                       ...                    ...                     ...   
859                      NaN                    NaN                     NaN   
860                      NaN                    NaN                     NaN   
861                      NaN                    NaN                     NaN   
862                      NaN                    NaN                     NaN   
863                      NaN                    NaN                     NaN   

     Italy          US  Spain  India     Germany  date  
0     20.0        11.0   13.0   28.0        12.0     0  
1     62.0        11.0   15.0   30.0        12.0     1  
2    155.0        11.0   32.0   31.0        12.0     2  
3    229.0        12.0   45.0   34.0        12.0     3  
4    322.0        12.0   84.0   39.0        13.0     4  
..     ...         ...    ...    ...         ...   ...  
859    NaN  85506177.0    NaN    NaN  26803867.0   859  
860    NaN  85515529.0    NaN    NaN  26809245.0   860  
861    NaN  85632808.0    NaN    NaN  26915085.0   861  
862    NaN  85758638.0    NaN    NaN  27007429.0   862  
863    NaN  85941735.0    NaN    NaN  27096571.0   863  

[864 rows x 9 columns]

In [62]:
quick_plot(pd_sync_timelines_w_slope.date,
           pd_sync_timelines_w_slope.iloc[:,0:5],
           y_scale='log',
           slider=True)

In [63]:
pd_sync_timelines_w_slope.to_csv('../data/processed/COVID_small_sync_timeline_table.csv',sep=';',index=False)

# Linear Regression

In [74]:
#importing libraries for scikitlearn
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=False)

In [92]:
#prepare entry matrices X,y (X-->input,y-->label)

l_vec=len(df_analyse['Germany']) #to have same number of rows in germany column vector
X=np.arange(l_vec-5).reshape(-1,1) #to maintain same number of rows and a single column as in germany #row means samples and cols means features
#y=np.array(df_analyse['Germany'][5:]) #here our target vector is to estimate the spread in germany
y=np.log(np.array(df_analyse['Germany'][5:])) #for log scale conversion

In [93]:
#training
reg.fit(X,y)

LinearRegression(fit_intercept=False)

In [94]:
#prediction
X_hat=np.arange(l_vec).reshape(-1, 1)
Y_hat=reg.predict(X_hat)

In [95]:
#pushing into a data frame
LR_inspect=df_analyse[['date','Germany']].copy()

In [98]:
LR_inspect['prediction']=np.exp(Y_hat)

In [100]:
quick_plot(LR_inspect.date,
           LR_inspect.iloc[:,1:],
           y_scale='log',
           slider=True)

# Doubling Rate - Piecewise Linear Regression

In [101]:
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)

In [102]:
from scipy import signal

In [113]:
#inspecting on small dataset
reg = linear_model.LinearRegression(fit_intercept=True)
l_vec=len(df_analyse['Germany'])
x=np.arange(l_vec-5).reshape(-1,1)
y=np.array(df_analyse['Germany'][5:])

In [114]:
reg.fit(X,y)

LinearRegression()

In [108]:
reg.intercept_

-4850069.374845953

In [115]:
reg.coef_

array([23143.53728042])

In [116]:
reg.coef_/reg.intercept_

array([-0.0047718])

In [124]:
def get_rate_via_regression(in_array):
    """" use a linear regression to approximate the slope """"""
    ----
    """
    
    y=np.array(in_array)
    X=np.arange(-1,2).reshape(-1,1)  #just 3 datapoints -1,0,1
    
    assert len(in_array)==3
    
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_
   
    return intercept/slope



In [126]:
#rolling command move the window across time series
(df_analyse['Germany']).rolling(window=3,min_periods=3).apply(get_rate_via_regression)

0              NaN
1              NaN
2              NaN
3              NaN
4              NaN
          ...     
871     819.813827
872    9968.635056
873     482.704820
874     271.571735
875     297.613719
Name: Germany, Length: 876, dtype: float64

In [128]:
#pushing it into a dataframe
df_analyse['Germany_DR']=df_analyse['Germany'].rolling(window=3,min_periods=3).apply(get_rate_via_regression)

In [133]:
quick_plot(df_analyse.date,df_analyse.iloc[40:200,[6]],y_scale='linear') #from 40days to 200 days

# Doubling time

In [134]:
#writing a function for doubling time
def doubling_time(in_array):
    " use a classical doubling time formular,see wikipidea for doubling time"
    y =np.array(in_array)
    return len(y)*np.log(2)/np.log(y[-1]/y[0])

In [135]:
#pushing it into a dataframe
df_analyse['Germany_DT_wiki']=df_analyse['Germany'].rolling(window=3,min_periods=3).apply(doubling_time)

In [136]:
#plotting doubling time
quick_plot(df_analyse.date,df_analyse.iloc[40:200,[6,7]],y_scale='linear') 

In [137]:
#now inspecting for more countries
country_list=df_analyse.columns[1:]
for each in country_list:
    df_analyse[each+'_DR']=df_analyse[each].rolling(window=3,min_periods=3).apply(get_rate_via_regression)

In [154]:
quick_plot(df_analyse.date,df_analyse.iloc[60:200,[6,7,8,9,10]],y_scale='linear',slider=True)  #plotting more countries by selecting more cols in dataframe as list

Here data is too bumpy. so we need to filter this to obtain a smooth representation

# Filtering 

In [155]:
#avoids signal trend distortion on a small window so that actual trend wont be lost
from scipy import signal

In [156]:
#Analysing on one dataset i.e. US
df_analyse['US'+'_filter']=signal.savgol_filter(df_analyse['US'],
                                                3,#window size for filtering
                                                1) #order of fitting polynomial

In [160]:
#plotting on us
start_pos=5
quick_plot(df_analyse.date[start_pos:],
           df_analyse[['US','US_filter']].iloc[start_pos:100,:],y_scale='log',slider=True)

In [175]:
df_analyse=pd.read_csv('../data/processed/COVID_small_flat_table.csv',sep=';',
                       parse_dates=[0])  
country_list=df_analyse.columns[1:]

In [176]:
## filter data
for each in country_list:
    df_analyse[each+'_filter']=signal.savgol_filter(df_analyse[each],
                           3, # window size used for filtering
                           1) # order of fitted polynomial
    

In [177]:
filter_cols=['Italy_filter','US_filter', 'Spain_filter', 'Germany_filter']


In [178]:
start_pos=5
quick_plot(df_analyse.date[start_pos:],
           df_analyse[filter_cols].iloc[start_pos:,:], #['US','US_filter']
           y_scale='log',
           slider=True)

In [179]:
#calculating slopes now
#inspecting data
df_analyse.head()

date  Italy  US  Spain  India  Germany  Italy_filter  US_filter  \
0 2020-01-22      0   1      0      0        0           0.0   0.833333   
1 2020-01-23      0   1      0      0        0           0.0   1.333333   
2 2020-01-24      0   2      0      0        0           0.0   1.666667   
3 2020-01-25      0   2      0      0        0           0.0   3.000000   
4 2020-01-26      0   5      0      0        0           0.0   4.000000   

   Spain_filter  India_filter  Germany_filter  
0           0.0           0.0        0.000000  
1           0.0           0.0        0.000000  
2           0.0           0.0        0.000000  
3           0.0           0.0        0.000000  
4           0.0           0.0        0.333333

In [180]:
def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''
    
    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)
    
    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_
    
    return intercept/slope

In [181]:
def doubling_time(in_array):
    ''' Use a classical doubling time formular, 
     see https://en.wikipedia.org/wiki/Doubling_time '''
    y = np.array(in_array)
    return len(y)*np.log(2)/np.log(y[-1]/y[0])

In [182]:
# calculate slope of regression of last x days
# use always a limited number of days to approximate the triangle, attention exponential base assumption
days_back = 3 # this gives a smoothing effect
for pos,country in enumerate(country_list):
    df_analyse[country+'_DR']=df_analyse[country].rolling(
                                window=days_back,
                                min_periods=days_back).apply(get_doubling_time_via_regression, raw=False)

In [183]:
# run on all filtered data
days_back = 3 # this gives a smoothing effect
for pos,country in enumerate(filter_cols):
    df_analyse[country+'_DR']=df_analyse[country].rolling(
                                window=days_back,
                                min_periods=days_back).apply(get_doubling_time_via_regression, raw=False)

In [184]:
# cross check the matematical 
df_analyse['Germany_DR_math']=df_analyse['Germany'].rolling(
                                window=days_back,
                                min_periods=days_back).apply(doubling_time, raw=False)

In [185]:
# run on all filtered data
days_back = 3 # this gives a smoothing effect
for pos,country in enumerate(filter_cols):
    df_analyse[country+'_DR']=df_analyse[country].rolling(
                                window=days_back,
                                min_periods=days_back).apply(get_doubling_time_via_regression, raw=False)

In [186]:
df_analyse.columns

Index(['date', 'Italy', 'US', 'Spain', 'India', 'Germany', 'Italy_filter',
       'US_filter', 'Spain_filter', 'India_filter', 'Germany_filter',
       'Italy_DR', 'US_DR', 'Spain_DR', 'India_DR', 'Germany_DR',
       'Italy_filter_DR', 'US_filter_DR', 'Spain_filter_DR',
       'Germany_filter_DR', 'Germany_DR_math'],
      dtype='object')

In [193]:
start_pos=40
quick_plot(df_analyse.date[start_pos:],
           df_analyse.iloc[start_pos:200,[11,12,13,14]], #
           y_scale='linear',
           slider=True)

In [194]:
## filter data
for each in country_list:
    df_analyse[each+'_filter']=signal.savgol_filter(df_analyse[each],
                           3, # window size used for filtering
                           1) # order of fitted polynomial

In [203]:
start_pos=5
quick_plot(df_analyse.date[start_pos:],
           df_analyse[filter_cols].iloc[start_pos:,:], 
           y_scale='linear',
           slider=True)